In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import yaml
from src.antares_io import load_api_config_from_yaml, update_variant_ids_in_config, prepare_data_for_batch
from src.antares_runner import AntaresRunner
from src.antares_reader import get_outputs_from_runner

In [ ]:
# --- Cell 2: Load Configs and Prepare Data Batch ---
CONFIG_LOCAL_PATH = Path("config_local.yaml")
CONFIG_RUN_PATH = Path("config_run.yaml")
CONFIG_ANTARES_PATH = Path("config_antares.yaml")

# Load all configurations into a single dictionary
run_config = yaml.safe_load(open(CONFIG_RUN_PATH))
run_config['antares_config'] = yaml.safe_load(open(CONFIG_ANTARES_PATH))
run_config['local_config'] = yaml.safe_load(open(CONFIG_LOCAL_PATH))

# Prepare the entire data batch with one function call
year_data_batch = prepare_data_for_batch(run_config)



In [ ]:
# --- Cell 3: Execute the Runner ---
# 1. Load the API config
api_config = load_api_config_from_yaml(CONFIG_LOCAL_PATH)

# 2. Initialize the Runner
runner = AntaresRunner(
    api_config=api_config,
    study_version=run_config['study_parameters']['antares_version'],
    solver_name="xpress",
    nb_cpu=34
)

In [ ]:
# 3. Launch the Batch Run
final_study_ids = runner.run_batch(
    year_data=year_data_batch,
    study_name_prefix=run_config['study_parameters']['study_name_prefix']
)

# 4. Save the new/updated IDs back to the config file
if final_study_ids:
    update_variant_ids_in_config(CONFIG_RUN_PATH, final_study_ids)

# 5. Wait for All Simulations to Finish
runner.wait_for_completion()

# 6. Collect and Display Results
outputs_by_year = get_outputs_from_runner(runner)
print("\n--- Final Consolidated Results ---")
print(outputs_by_year)

In [10]:
outputs_by_year = get_outputs_from_runner(runner)


--- Fetching All Simulation Outputs from API ---
Fetching output for year 2030 (Study: 'HH-SL-0.1.1_run_2030', Output ID: 20250915-1743eco)...
Fetching output for year 2040 (Study: 'HH-SL-0.1.1_run_2040', Output ID: 20250915-1747eco)...
Fetching output for year 2050 (Study: 'HH-SL-0.1.1_run_2050', Output ID: 20250915-1752eco)...
Successfully fetched 3 output(s).


In [ ]:

VALUES  = general
DETAILS = thermal plants

DETAILS_ST_STORAGE = st_storages



outputs_by_year[2030].get_mc_all_area(frequency=Frequency.HOURLY, data_type=MCIndAreasDataType.DETAILS_ST_STORAGE , area= 'fr_el').columns
MultiIndex([('battery',  'P-injection - MW', 'EXP'),
            (    'dam',  'P-injection - MW', 'EXP'),
            (    'phs',  'P-injection - MW', 'EXP'),
            ('battery', 'P-withdrawal - MW', 'EXP'),
            (    'dam', 'P-withdrawal - MW', 'EXP'),
            (    'phs', 'P-withdrawal - MW', 'EXP'),
            ('battery',      'Levels - MWh', 'EXP'),
            (    'dam',      'Levels - MWh', 'EXP'),
            (    'phs',      'Levels - MWh', 'EXP'),
            ('battery',   'CashFlow - Euro', 'EXP'),
            (    'dam',   'CashFlow - Euro', 'EXP'),
            (    'phs',   'CashFlow - Euro', 'EXP')],
           )

outputs_by_year[2030].get_mc_all_area(frequency=Frequency.HOURLY, data_type=MCIndAreasDataType.DETAILS , area= 'fr_el').columns
MultiIndex([(    'ccgt',            'MWh', 'EXP'),
            ('new_nuke',            'MWh', 'EXP'),
            (    'ocgt',            'MWh', 'EXP'),
            ('old_nuke',            'MWh', 'EXP'),
            (    'ccgt',  'MIN GEN - MWh', 'EXP'),
            ('new_nuke',  'MIN GEN - MWh', 'EXP'),
            (    'ocgt',  'MIN GEN - MWh', 'EXP'),
            ('old_nuke',  'MIN GEN - MWh', 'EXP'),
            (    'ccgt', 'NP Cost - Euro', 'EXP'),
            ('new_nuke', 'NP Cost - Euro', 'EXP'),
            (    'ocgt', 'NP Cost - Euro', 'EXP'),
            ('old_nuke', 'NP Cost - Euro', 'EXP'),
            (    'ccgt',           'NODU', 'EXP'),
            ('new_nuke',           'NODU', 'EXP'),
            (    'ocgt',           'NODU', 'EXP'),
            ('old_nuke',           'NODU', 'EXP'),
            (    'ccgt',  'Profit - Euro', 'EXP'),
            ('new_nuke',  'Profit - Euro', 'EXP'),
            (    'ocgt',  'Profit - Euro', 'EXP'),
            ('old_nuke',  'Profit - Euro', 'EXP')],
           )

outputs_by_year[2030].get_mc_all_area(frequency=Frequency.HOURLY, data_type=MCIndAreasDataType.VALUES , area= 'fr_el').columns
MultiIndex([(  'OV. COST', 'Euro', 'EXP'),
            (  'OP. COST', 'Euro', 'EXP'),
            (  'OP. COST', 'Euro', 'std'),
            (  'OP. COST', 'Euro', 'min'),
            (  'OP. COST', 'Euro', 'max'),
            ('MRG. PRICE', 'Euro', 'EXP'),
            ('MRG. PRICE', 'Euro', 'std'),
            ('MRG. PRICE', 'Euro', 'min'),
            ('MRG. PRICE', 'Euro', 'max'),
            ( 'CO2 EMIS.', 'Tons', 'EXP'),
            ...
            (   'MAX MRG',  'MWh', 'min'),
            (   'MAX MRG',  'MWh', 'max'),
            (   'NP COST', 'Euro', 'EXP'),
            (   'NP COST', 'Euro', 'std'),
            (   'NP COST', 'Euro', 'min'),
            (   'NP COST', 'Euro', 'max'),
            (      'NODU',    ' ', 'EXP'),
            (      'NODU',    ' ', 'std'),
            (      'NODU',    ' ', 'min'),
            (      'NODU',    ' ', 'max')],
           length=253)

In [ ]:
# outputs_by_year[2030].aggregate_mc_all_areas(data_type="details", frequency="hourly")
from antares.craft.model.output import MCIndAreasDataType, MCAllAreasDataType

# outputs_by_year[2030].get_mc_all_area(frequency=Frequency.HOURLY, data_type=MCIndAreasDataType.VALUES , area= 'fr_el').columns

outputs_by_year[2030].aggregate_mc_all_areas(frequency=Frequency.HOURLY, data_type=MCIndAreasDataType.VALUES)


In [ ]:
from enum import Enum
class MCIndAreasDataType(Enum):
    VALUES = "values"
    DETAILS = "details"
    DETAILS_ST_STORAGE = "details-STstorage"
    DETAILS_RES = "details-res"
class Frequency(Enum):
    HOURLY = "hourly"
    # DAILY = "daily"
    # WEEKLY = "weekly"
    # MONTHLY = "monthly"
    # ANNUAL = "annual"



In [ ]:
MCIndAreasDataType

In [ ]:
# TODO test the accurate mod
# TODO the settings for management policy + weather_year selection ...

In [ ]:
# TODO: Add a cell to run study
# TODO: Add a cell to read results
# TODO: Filter results saved

# TODO: factor study update in a configurable function
# TODO: Add h2 import
# TODO: add salt_cavern
# TODO: forward availability for nuke / old_nuke conversion_tech
# TODO: forward mustrun for nuke / old_nuke conversion_tech  -->> Defined per cluster in antares, not possible here
# TODO:
# TODO: add parameters not yet implemented

# TODO: clean up conf files
# TODO: do a propper logger
# TODO: skip part of the processes that are not necessary  after the study has been initialised (e.g. it 3, no need to send VRES load_factor (defined in % and so normalised) again, as they are already properly defined and will not be affected by invenstment changes, likely the same for hydro as they can't be invested
# TODO: check if it works for 2030, 2040 and 2050
# TODO: read directly pommes inputs file
